In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob 
import json
import re
import string
import operator
import collections
import numpy as np
import string
import math
from tqdm import tqdm
from array import array
import pandas as pd

In [2]:
NONE_STR = '<s>'

In [3]:
def delete_other_letter(line):
    punctuation ="""!"#$%&\()*+,./:;<=>?@[\\]^_`{|}~\\n"""
    re_punctuation ="[{}]+".format(punctuation)
    line =re.sub(re_punctuation, "", line)
    return line.lower()

In [4]:
def read_articles(paths):
    alllines = []
    for path in paths:
        filenames = glob.glob(path+"/*txt")
        for filename in filenames:
            with open(filename, 'r', encoding='utf-8') as fpr:
                data_raw = json.load(fpr)
                article = data_raw['article']
                
                lines = article.split('.')
                for line in lines:
                    line = delete_other_letter(line)
                    alllines.append(line)
                
    return alllines

In [5]:
lines = read_articles(['./RACE/train/high','./RACE/train/middle'])

In [6]:
print(len(lines))

499388


In [7]:
# def word_dict(lines):
#     word_freq = collections.defaultdict(int)
#     for line in lines:
#         for w in line.split(): 
#             word_freq[w] += 1
    
#     return list(word_freq.keys())

In [8]:
# def get_words_matrix(words,lines):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
#     for line in lines:
#         line_words = line.split()
#         n = len(line_words)
#         for i in range(n-1):
#             ww = line_words[i+1]
#             i = words.index(line_words[i])
#             j = words.index(ww)
#             words_matrix[i][j] += 1

In [9]:
def word_dict(lines,n):
                
    pre_words = collections.defaultdict(int)
    be_words = collections.defaultdict(int)
    n = n-1
    for line in lines:
        for k in range(n):
            line = NONE_STR+' '+line+' '+NONE_STR
        line_words = line.split()
        for i in range(len(line_words)-(n-1)):
            ngramTemp = ' '.join(line_words[i:i+n])

            pre_words[ngramTemp] += 1
            be_words[line_words[i]] += 1
            
    return pre_words,be_words

In [10]:
def replace_punctuation(word):
    for puc in punctuation_list:
        word = word.replace(puc,'')
    return word

In [11]:
def replace_Chinese_punctuation(content):
    content = content.replace('’','\'')
    content = content.replace('”','')
    return content

In [12]:
def get_options(options_str):
    options_str = replace_Chinese_punctuation(options_str)
    options = []
    opt_rows = options_str.split('#')
    for opt in opt_rows:
        indexA = opt.find('A.')
        indexB = opt.find('B.')
        indexC = opt.find('C.')
        indexD = opt.find('D.')
        As = opt[indexA+2:indexB].strip().lower().split(' ')
        Bs = opt[indexB+2:indexC].strip().lower().split(' ')
        Cs = opt[indexC+2:indexD].strip().lower().split(' ')
        Ds = opt[indexD+2:].strip().lower().split(' ')
        options.append([As,Bs,Cs,Ds])
    return options

In [13]:
# def probability(word1,word2):
#     eclipse = 0.0000000000001
#     try:
#         index1 = pre_words.index(word1)
#         index2 = be_words.index(word2)
#     except ValueError:
#         return eclipse
# #     print(word1,index1,word2,index2)
#     pn_word1 = words_matrix[index1].sum()
#     pn_word2_under_word1 = words_matrix[index1][index2]
# #     print(pn_word1,pn_word2_under_word1)
#     prob = float(pn_word2_under_word1)/pn_word1
#     if prob == 0:
#         prob = eclipse
#     return prob

In [14]:
def read_train_data(n):
    path = './data'
    filenames = glob.glob(path+"/*txt")
    
    answers_list =[]
    options_list = []
    content_list = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as fpr:
            data_raw = json.load(fpr)
            article = data_raw['article']
            content = get_pre_and_be_words(article,n)
            options_str = data_raw['options']
            options = get_options(options_str)
            answers = list(data_raw['answers'])
            answers_list.append(answers)
            options_list.append(options)
            content_list.append(content)
    return content_list,options_list,answers_list

In [15]:
def test(content_list,options_list,answers_list,ngram,showAll=False,predict=None):
    
    cn = len(content_list)
    right_n = 0
    total_n = 0
    for k in range(cn):
        content = content_list[k]
        answers = answers_list[k]
        options = options_list[k]
    
        n = len(answers)
        total_n += n
        for i in range(n):
            if showAll:
                print('---------'+str(i+1)+'---------')
            if predict is None:
#                 print('before:',content[i])
                choice = auto_select(content[i],options[i],ngram)
#                 print('after:',content[i])
            else:
                choice = predict
            if choice == answers[i]:
                right_n += 1
                if showAll == False:
                    print('---------'+str(i+1)+'---------')
                    print('predict:',choice,'ground truth:',answers[i])
                    print('空前后的词',content[i])
                    print('选项',options[i])
            if showAll:
                print('predict:',choice,'ground truth:',answers[i])
                print('空前后的词',content[i])
                print('选项',options[i])
    accuracy = float(right_n)/total_n
    print('accuracy:',accuracy)

In [16]:
def get_words_matrix(pre_words,be_words,lines,n):  
    n = n-1
    for line in tqdm(lines):
        for k in range(n):
            line = NONE_STR+' '+line+' '+NONE_STR
        line_words = line.split()
        words_n = len(line_words)-n
        for k in range(words_n):
            pre_word = ' '.join(line_words[k:k+n])
                
            be_word = line_words[k+n]
            
            if pre_word in pre_words and be_word in be_words:
                i = pre_words.index(pre_word)
                j = be_words.index(be_word)
                words_matrix[i][j] += 1
                    
#             if n > 1:
#                 if pre_word in pre_words:
#                     i = pre_words.index(pre_word)
#                     j = be_words.index(be_word)
#                     words_matrix[i][j] += 1
#             else:
#                 i = pre_words.index(pre_word)
#                 j = be_words.index(be_word)
#                 words_matrix[i][j] += 1
            
    return words_matrix

In [17]:
def count_word_num(lines,n):
    pre_words = collections.defaultdict(int)
    be_words = collections.defaultdict(int)
    
    words = collections.defaultdict(int)
    n = n-1
    
    index = 0
    for line in lines:
        for k in range(n):
            line = NONE_STR+' '+line+' '+NONE_STR
        line_words = line.split()
        for i in range(len(line_words)-(n-1)):
            pre_word = ' '.join(line_words[i-1:i+n-1])
            be_word = line_words[i+n-1]
            
            pre_words[pre_word] += 1
            be_words[be_word] += 1
            
            words[pre_word+' '+be_word] += 1
    return pre_words,be_words,words

In [18]:
punctuation_list = ['.',',','?','!','\"','“']

In [19]:
def check_punctuation_and_replace_with_none(word_list,reverse = False):
    n = len(word_list)
    f_flag = False #第一个字符是否为标点
    b_flag = False #最后一个字符是否为标点
    if reverse == True:
        for i in range(n-1, -1, -1):
            word = word_list[i]
            if b_flag == False:
                for puc in punctuation_list:
                    if word[-1]==puc :
                        b_flag = True 
                        break
                    elif word[0] == puc:
                        f_flag = True
                        break
            if b_flag == True:
                word_list[i] = NONE_STR
            else:
                word_list[i] = replace_punctuation(word).lower()
            if f_flag == True:
                b_flag = True
    else:
        for i in range(n):
            word = word_list[i]
            if f_flag == False:
                for puc in punctuation_list:
                    if word[0] == puc:
                        f_flag = True
                        break
                    elif word[-1]==puc :
                        b_flag = True 
                        break
            if f_flag == True:
                word_list[i] = NONE_STR
            else:
                word_list[i] = replace_punctuation(word).lower()
            if b_flag == True:
                f_flag = True
    return word_list

In [20]:
def get_pre_and_be_words(article,ngram):
    article = replace_Chinese_punctuation(article)
    art_words = article.split()
    n = len(art_words)
    index = 1
    content = []
    for i in range(n):
        word = art_words[i]
        if str(index) == word:
            pre_word_list = []
            be_word_list = []
            if i < ngram-1:
                for k in range(ngram-1-i):
                    pre_word_list.append(NONE_STR)
            else:
                pre_word_list = art_words[i-1-(ngram-2):i]
            pre_word_list = check_punctuation_and_replace_with_none(pre_word_list,True)

            if i > n-ngram:
                for k in range(ngram-1-(n-ngram)):
                    be_word_list.append(NONE_STR)
            else:
                be_word_list = art_words[i+1:i+1+ngram-1]
            be_word_list = check_punctuation_and_replace_with_none(be_word_list,False)
            
            content.append([pre_word_list,be_word_list])
            index += 1
    return content

In [21]:
def delete_sparse_words(words_dict,threshold=4):
    words_list = []
    for key, value in words_dict.items():
        if value > 4:
            words_list.append(key)
    return words_list

In [22]:
def probability(word1,word2):
    eclipse = 0.0000000000001
    all = pre_words[word1]
    single = words[word1+' '+word2]
    
    if all == 0:
        return eclipse
    
    prob = float(single)/all
    if prob == 0:
        prob = eclipse
        
    return prob

<font color=blue size=15 face=雅黑>bigram Test</font>

In [ ]:
pre_words,be_words,words = count_word_num(lines,2)

In [ ]:
content_list,options_list,answers_list = read_train_data(2)

In [ ]:
print(content_list[0])

In [ ]:
print(options_list[0])

In [ ]:
print(answers_list[0])

<font color=black size=5 face=雅黑>全选A/B/C/D 观察正确率</font>

In [ ]:
test(content_list,options_list,answers_list,2,showAll=False,predict='A')

In [ ]:
test(content_list,options_list,answers_list,2,showAll=False,predict='B')

In [ ]:
test(content_list,options_list,answers_list,2,showAll=False,predict='C')

In [ ]:
test(content_list,options_list,answers_list,2,showAll=False,predict='D')

<font color=black size=5 face=雅黑>采用bigram模型后，正确率</font>

In [ ]:
def auto_select(question,option,ngram):
    index = 0
    maxprob = -10000000000
    n = len(option)
    for i in range(n):
        word_list = []
        word_list = question[0].copy()
        word_list.append(option[i][0])
        word_list.extend(question[1])
        wln = len(word_list)
        prob = 0
        for k in range(wln-ngram+1):
            pre_word = ' '.join(word_list[k:k+ngram-1])
            prob += math.log(probability(pre_word,word_list[k+ngram-1]))
        if prob > maxprob:
            maxprob = prob
            index = i
    return chr(ord('A')+index)

In [ ]:
print(content_list[0])

In [ ]:
test(content_list,options_list,answers_list,2,showAll=True)

In [ ]:
test(content_list,options_list,answers_list,2,showAll=False)

<font color=blue size=15 face=雅黑>trigram Test</font>

In [ ]:
N = 3
pre_words,be_words,words = count_word_num(lines,N)

In [ ]:
content_list,options_list,answers_list = read_train_data(N)

In [ ]:
test(content_list,options_list,answers_list,N,showAll=True)

In [ ]:
test(content_list,options_list,answers_list,N,showAll=False)

In [ ]:
tt = 'has a'
print(pre_words[tt])

In [ ]:
n = len(tt)
for key, value in words.items():
#     print(key[:n])
    if key[:n] == tt and key[n]==' ':
        print(key,' : ',value)

<font color=blue size=15 face=雅黑>fourgram Test</font>

In [ ]:
N = 4
pre_words,be_words,words = count_word_num(lines,N)
content_list,options_list,answers_list = read_train_data(N)
test(content_list,options_list,answers_list,N,showAll=True)

In [ ]:
test(content_list,options_list,answers_list,N,showAll=False)